In [ ]:
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import pandas as pd
import getpass
import os
from IPython.display import Image, display
import glob
import json
root = 'https://deslabs.ncsa.illinois.edu/easyweb/api'

### Authenticate 

In [ ]:
user = 'mcarras2'
passwd = getpass.getpass()

### Create a new token
This will be used to submit jobs, it can be re-created any time

In [ ]:
post_data = {
    'username': user,
    'password': passwd
}

In [ ]:
req = requests.post(root+'/v1/token/',data=post_data, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
token = req.json()['token']
print(f'Token: {token}')

### Check Token status

In [ ]:
req = requests.get(root+'/v1/token/?token={}'.format(token), verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))

### Sumbit a Query Job

In [ ]:
#query
q = "select ra,dec from y3a2_coadd_object_summary where rownum < 100"
# create body of POST request
body = {
    'token'       : token,
    'query'       :  q,
    'db'          : 'dessci',
    'output'      : 'test.csv',
    'compression' : 'no',
    'email'       : '',
    'jobname'     : 'api_test2'
}

In [ ]:
req = requests.post(root+'/v1/query/', data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
job = req.json()['jobid']
print(f'Job id: {job}')

### Check Job by its id

In [ ]:
body = {
    'token'       : token,
    'jobid'       : job
}

In [ ]:
req = requests.post(root+'/v1/jobs/', data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))

In [ ]:
for f  in req.json()['files']:
    print(f)

### Check all Jobs

In [ ]:
body = {
    'token'       : token,
    'jobid'       : 'all'
}

In [ ]:
req = requests.post(root+'/v1/jobs/', data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))

In [ ]:
df = pd.DataFrame.from_dict(req.json())
df = df.drop(['msg', 'status'], axis=1)

In [ ]:
df

### Delete a Job

In [ ]:
body = {
    'token'       : token,
    'jobid'       : job
}

In [ ]:
req = requests.delete(root+'/v1/jobs/', data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))

### Delete All Jobs

In [ ]:
body['jobid'] = 'all'
all_jobs = requests.post(root+'/v1/jobs/', data=body, verify=False)

### !! Uncomment these lines if you really want to delete all jobs

In [ ]:
# for j in range(len(all_jobs.json()['job_id'])):
#     body['jobid'] = all_jobs.json()['job_id'][j]
#     req = requests.delete(root+'/v1/jobs/', data=body, verify=False)
#     print(req.text)

## Cutouts (with ra/dec input list)

In [ ]:
ra = [27.982411, 22.922076, 56.337698]
dec = [-43.656078, -42.093011, -61.875547]
xs = 1.0
ys = 1.0
# create body of request
body = {
    'token'            :  token,        # required
    'ra'               : str(ra),       # required of coadd/coord/csvfile
    'dec'              : str(dec),      # required of coadd/coord/csvfile
    'xsize'            : str(xs),       # required (default : 1.0)
    'ysize'            : str(ys),       # required (default : 1.0)
    'jobname'          : 'test ra/dec list',        # optional (default : 'false') 'true': will not generate pngs (faster)
    'make_stiff_rgb'   : 'true',       # optional
    'make_lupton_rgb'  : 'true',        # optional
    'make_fits'        : 'false',       # optional
    'rgb_values'       : 'i,r,g',
    'bands'            : 'g,r,i,z,y',   # required if make_fits is true; pick any or all bands
}
req = requests.post(root+'/v1/cutout/',data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
job = req.json()['jobid']

In [ ]:
body = {'token':token,'jobid':job}
req = requests.post(root+'/v1/jobs/', data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))

In [ ]:
list_pngs = glob.glob('../../jobs/{0}/*/*.png'.format(job))
for i in range(len(list_pngs)):
    x = Image(list_pngs[i])
    display(x)

## Cutouts (with coadd input list)

In [ ]:
coadd = [192849936, 267703666]
xs = 1.0
ys = 1.0
# create body of request
body = {
    'token'            :  token,        # required
    'coadd'        : str(coadd),    # required of coadd/coord/csvfile
    'xsize'            : str(xs),       # required (default : 1.0)
    'ysize'            : str(ys),       # required (default : 1.0)
    'jobname'          : 'test ra/dec list',        # optional (default : 'false') 'true': will not generate pngs (faster)
    'make_stiff_rgb'   : 'false',       # optional
    'make_lupton_rgb'  : 'true',        # optional
    'make_fits'        : 'false',       # optional
    'rgb_values'       : 'i,r,g',
    'bands'            : 'g,r,i,z,y',   # required if make_fits is true; pick any or all bands
}
req = requests.post(root+'/v1/cutout/',data=body, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
job = req.json()['jobid']

## Cutouts (with CSV input)

In [ ]:
#Lets get a file first
#query
q = "select ra,dec from y3a2_coadd_object_summary where rownum < 5"
# create body of POST request
body = {
    'token'       : token,
    'query'       :  q,
    'db'          : 'dessci',
    'output'      : 'test.csv',
    'compression' : 'no',
    'email'       : '',
    'jobname'     : 'api_test'
}
req = requests.post(root+'/v1/query/', data=body, verify=False)
print('msg  : {}'.format(req.text))
job = req.json()['jobid']
print(f'Job id: {job}')

In [ ]:
body = {'token': token,'jobid': job}
req = requests.post(root+'/v1/jobs/', data=body, verify=False)
print('msg  : {}'.format(req.text))
for f_url  in req.json()['files']:
    print(f_url)
    filename = os.path.basename(f_url)
    r = requests.get(f_url)
    with open(filename, 'wb') as f:  
        f.write(r.content)

In [ ]:
csvfile = 'test.csv'
files = {'csvfile' : ('cf', open(csvfile, 'rb'), 'text/csv', {'Expires':'0'})}
xs = 1.0
ys = 1.0
# create body of request
body = {
    'token'            :  token,        # required
    'csvfile'      :  csvfile,      # required of coadd/coord/csvfile
    'xsize'            : str(xs),       # required (default : 1.0)
    'ysize'            : str(ys),       # required (default : 1.0)
    'jobname'          : 'test ra/dec list',        # optional (default : 'false') 'true': will not generate pngs (faster)
    'make_stiff_rgb'   : 'false',       # optional
    'make_lupton_rgb'  : 'true',        # optional
    'make_fits'        : 'false',       # optional
    'rgb_values'       : 'i,r,g',
    'bands'            : 'g,r,i,z,y',   # required if make_fits is true; pick any or all bands
}
req = requests.post(root+'/v1/cutout/',data=body, files=files, verify=False)
print('url  : {}\n'.format(req.url))
print('code : {}\n'.format(req.status_code))
print('msg  : {}'.format(req.text))
job = req.json()['jobid']